<a href="https://colab.research.google.com/github/alexanderBennington/BETO_cardiologia/blob/main/cardiologia_29_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets pandas onnx

In [ ]:
# Importa pandas para manejar el archivo CSV, Dataset para crear el dataset,
# y los módulos de transformers para tokenización, modelo, entrenamiento y pipeline
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, pipeline

In [ ]:
# Define el nombre del modelo y carga el tokenizador y el modelo de transformers
model_name = "dccuchile/bert-base-spanish-wwm-cased"
#model_name = "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Carga el tokenizador para el modelo
model = AutoModelForQuestionAnswering.from_pretrained(model_name)  # Carga el modelo preentrenado para preguntas-respuestas

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Cargar el archivo CSV y verificar si la ruta es correcta
try:
    data = pd.read_csv('cardiologia.csv', encoding="utf-8")  # Carga el archivo en un DataFrame
    print(data.head())  # Muestra las primeras filas para verificar el contenido
except FileNotFoundError:
    print("El archivo no se encontró. Verifica la ruta.")  # Muestra un mensaje de error si el archivo no se encuentra

# Crear el dataset desde el DataFrame de pandas
dataset = Dataset.from_pandas(data)
print(dataset)

                                             context  \
0  El corazón está en el centro del pecho, cerca ...   
1  El corazón es un órgano que tiene el tamaño ap...   
2  El corazón es un órgano que tiene el tamaño ap...   
3  Las dos cámaras superiores del corazón se llam...   
4  Una pared interna de tejido divide los lados d...   

                                            question  \
0                            ¿Dónde esta el corazón?   
1                                ¿Qué es el corazón?   
2                   ¿Cuál es la función del corazón?   
3  ¿Cómo se llaman las camaras superiores del cor...   
4                                ¿Qué es el tabique?   

                                              answer  start_positions  \
0      en el centro del pecho, cerca de los pulmones               16   
1  es un órgano que tiene el tamaño aproximado de...               11   
2                   bombea sangre por todo el cuerpo               72   
3                                 

In [ ]:
# Preparar los datos en el formato adecuado para el modelo de pregunta-respuesta
def prepare_data(example):
    return {
        'context': example['context'],           # Extrae el contexto
        'question': example['question'],          # Extrae la pregunta
        'answers': {                              # Crea un diccionario para la respuesta
            'text': [example['answer']],       # Incluye la respuesta como lista
            'answer_start': [example['start_positions']]  # Posición de inicio de la respuesta en el contexto
        }
    }

# Aplica la función prepare_data a cada entrada del dataset
dataset = dataset.map(prepare_data)
#dataset = dataset.remove_columns(['start_positions', 'answer'])
print(dataset)
#print(dataset['answers'])

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'question', 'answer', 'start_positions', 'end_positions', 'answers'],
    num_rows: 225
})


In [ ]:
# Define una función para tokenizar cada ejemplo (pregunta y contexto)
def tokenize_examples(example):
    return tokenizer(
        example['question'],           # La pregunta a tokenizar
        example['context'],            # El contexto a tokenizar
        truncation=True,               # Recorta el texto si excede el límite de tokens
        max_length=384,                # Longitud máxima de tokens
        padding='max_length',          # Añade padding si es necesario
        return_offsets_mapping=True    # Devuelve el mapeo de offsets para ajustar etiquetas
    )

# Tokeniza el dataset aplicando la función tokenize_examples a cada ejemplo
tokenized_dataset = dataset.map(tokenize_examples, batched=True)
print(tokenized_dataset)

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'question', 'answer', 'start_positions', 'end_positions', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'],
    num_rows: 225
})


In [ ]:
#print(tokenized_dataset[0])
# Obtiene los IDs de tokens de un ejemplo y conviértelos a palabras legibles
example_tokens = tokenized_dataset[4]['input_ids']
print(tokenizer.convert_ids_to_tokens(example_tokens),'\n')
print(tokenized_dataset[4]['input_ids'],'\n')#Son IDs que corresponden a cada token, son los valores numéricos que el modelo utiliza internamente en lugar de las palabras.
print(tokenized_dataset[4]['attention_mask'],'\n')#Tiene valores 1 para los tokens relevantes (los de la pregunta y contexto) y 0 para el padding
print(tokenized_dataset[4]['offset_mapping'],'\n')#Muestra indice inicial y final de cada token en LN

['[CLS]', '¿', 'Qué', 'es', 'el', 'tab', '##ique', '?', '[SEP]', 'Una', 'pared', 'interna', 'de', 'tejido', 'divide', 'los', 'lados', 'derecho', 'e', 'izquierdo', 'del', 'corazón', '.', 'Esa', 'pared', 'se', 'llama', 'tab', '##ique', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [ ]:
print(tokenizer.convert_ids_to_tokens(4),'\n')

[CLS] 



In [ ]:
# Ajusta las etiquetas de inicio y fin de la respuesta dentro del contexto usando los offsets
def adjust_labels(example):
    offsets = example["offset_mapping"]
    #print(offsets)
    answer_start = example["answers"]["answer_start"][0]  # Posición inicial de la respuesta
    #print(example["answers"]["answer_start"][0])
    answer_end = answer_start + len(example["answers"]["text"][0])  # Calcula la posición final
    #print(answer_end)

    # Encuentra las posiciones de inicio y fin en la secuencia tokenizada
    start_position = end_position = None
    for idx, offset in enumerate(offsets):
        if offset[0] <= answer_start < offset[1]:  # Encuentra el índice de inicio
            start_position = idx
        if offset[0] < answer_end <= offset[1]:    # Encuentra el índice de fin
            end_position = idx
            break

    # Ajusta las posiciones si no se encuentran
    if start_position is None or end_position is None:
        start_position = 0 if start_position is None else start_position
        end_position = 0 if end_position is None else end_position

    # Asigna las posiciones de inicio y fin al ejemplo
    #print(start_position)
    example['start_positions'] = start_position
    example['end_positions'] = end_position

    #print(example)
    return example

# Aplica la función adjust_labels al dataset para calcular posiciones de respuestas
tokenized_dataset = tokenized_dataset.map(adjust_labels)

# Filtra ejemplos que no tengan posiciones válidas de respuesta
tokenized_dataset = tokenized_dataset.filter(lambda example: example['start_positions'] != 0 and example['end_positions'] != 0)
print(tokenized_dataset)

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Filter:   0%|          | 0/225 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'question', 'answer', 'start_positions', 'end_positions', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'],
    num_rows: 224
})


In [ ]:
# Define una función para verificar los offsets
def verify_offsets(example):
    context = example['context']
    question = example['question']
    answer = example['answers']['text'][0]
    offsets = example['offset_mapping']
    start_position = example['start_positions']
    end_position = example['end_positions']

    # Imprimir el contexto, la pregunta y la respuesta
    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Offsets: {offsets}")
    print(f"Start position: {start_position}, End position: {end_position}")

    # Verificar que los offsets correspondan a las posiciones correctas en el contexto
    token_start = offsets[start_position][0]
    token_end = offsets[end_position][1]
    extracted_answer = context[token_start:token_end]
    print(f"Extracted answer: {extracted_answer}")

    # Comparar la respuesta extraída con la respuesta original
    if extracted_answer == answer:
        print("Offsets are correct!")
    else:
        print("Offsets are incorrect!")

# Aplicar la función de verificación a algunos ejemplos del dataset tokenizado
for i in range(5):  # Verificar los primeros 5 ejemplos
    verify_offsets(tokenized_dataset[i])


Context: El corazón está en el centro del pecho, cerca de los pulmones
Question: ¿Dónde esta el corazón?
Answer: en el centro del pecho, cerca de los pulmones
Offsets: [[0, 0], [0, 1], [1, 6], [7, 11], [12, 14], [15, 22], [22, 23], [0, 0], [0, 2], [3, 10], [11, 15], [16, 18], [19, 21], [22, 28], [29, 32], [33, 38], [38, 39], [40, 45], [46, 48], [49, 52], [53, 61], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [

In [ ]:
# Divide el dataset en 80% para entrenamiento y 20% para validación
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']  # Conjunto de entrenamiento
val_dataset = train_test_split['test']     # Conjunto de validación
print(train_dataset)
print(val_dataset)

Dataset({
    features: ['context', 'question', 'answer', 'start_positions', 'end_positions', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'],
    num_rows: 179
})
Dataset({
    features: ['context', 'question', 'answer', 'start_positions', 'end_positions', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'],
    num_rows: 45
})


In [ ]:
# Configura los argumentos de entrenamiento del modelo
"""
training_args = TrainingArguments(
    output_dir='./results',            # Directorio donde se guardarán los resultados
    eval_strategy="epoch",             # Estrategia de evaluación en cada época
    learning_rate=2e-5,                # Tasa de aprendizaje baja
    per_device_train_batch_size=12,     # Tamaño de lote de entrenamiento
    per_device_eval_batch_size=12,      # Tamaño de lote de evaluación
    num_train_epochs=3,                # Número de épocas
    weight_decay=0.01,                 # Decaimiento de pesos para regularización
    report_to="none"                   # Desactiva el reporte a sistemas de seguimiento
)"""

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="steps",               # Cambia a "steps" para evaluación más frecuente
    logging_steps=50,                   # Frecuencia de logging (ajusta según tu dataset)
    learning_rate=5e-6,                  # Tasa de aprendizaje levemente ajustada
    per_device_train_batch_size=16,      # Reduce tamaño del lote para evitar problemas de memoria
    per_device_eval_batch_size=16,       # Ajusta también para evaluación
    num_train_epochs=3,                  # Empieza con menos épocas
    weight_decay=0.05,                   # Peso de decaimiento estándar
    report_to="none"
)

# Define el Trainer que manejará el proceso de entrenamiento
trainer = Trainer(
    model=model,                       # Modelo a entrenar
    args=training_args,                # Argumentos de entrenamiento
    train_dataset=train_dataset,       # Conjunto de datos para entrenamiento
    eval_dataset=val_dataset,          # Conjunto de datos para validación
    tokenizer=tokenizer                # Tokenizador usado
)

In [ ]:
# Iniciar el entrenamiento
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=36, training_loss=3.539603763156467, metrics={'train_runtime': 44.6015, 'train_samples_per_second': 12.04, 'train_steps_per_second': 0.807, 'total_flos': 105237268775424.0, 'train_loss': 3.539603763156467, 'epoch': 3.0})

In [ ]:
# Guardar el modelo y tokenizer
model.save_pretrained('./modelo-finetuned-cardiologia05')
tokenizer.save_pretrained('./modelo-finetuned-cardiologia05')

('./modelo-finetuned-cardiologia05/tokenizer_config.json',
 './modelo-finetuned-cardiologia05/special_tokens_map.json',
 './modelo-finetuned-cardiologia05/vocab.txt',
 './modelo-finetuned-cardiologia05/added_tokens.json',
 './modelo-finetuned-cardiologia05/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Cambia la ruta a donde deseas guardar el modelo en tu Google Drive
drive_path = '/content/drive/My Drive/modelo-finetuned-cardiologia05'

# Define la ruta local del modelo
local_model_path = './modelo-finetuned-cardiologia05'

# Copia el modelo y el tokenizador a Google Drive
# Agrega el argumento dirs_exist_ok=True para permitir la copia si el directorio existe
shutil.copytree(local_model_path, drive_path, dirs_exist_ok=True)

'/content/drive/My Drive/modelo-finetuned-cardiologiaQA'

In [ ]:
# Cargar el pipeline de pregunta-respuesta para pruebas
qa_pipeline = pipeline(
    'question-answering',
    model='./modelo-finetuned-cardiologia05',
    tokenizer='./modelo-finetuned-cardiologia05'
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Cargar el pipeline de pregunta-respuesta para pruebas
qa_pipeline = pipeline(
    'question-answering',
    model='mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
    tokenizer='mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Cargar el contenido de un archivo .txt como contexto
try:
    with open("contexto.txt", "r", encoding="utf-8") as file:
        context = file.read()
except UnicodeDecodeError:
    # Si falla utf-8, intenta con 'latin-1'
    with open("contexto.txt", "r", encoding="latin-1") as file:
        context = file.read()

In [ ]:
# Usar el contexto cargado en el pipeline de pregunta-respuesta
result = qa_pipeline({
    #'context': context,
    'context': """El corazón se compone de tres capas de tejido.
* El endocardio es la delgada capa interna de las cámaras del corazón y también forma la superficie de las válvulas.
* El miocardio es la gruesa capa muscular del medio, que permite que las cámaras del corazón se contraigan y se relajen para bombear sangre al cuerpo.
* El pericardio es el saco que envuelve el corazón. Está compuesto de capas delgadas de tejido y mantiene al corazón en su lugar y lo protege. Una pequeña cantidad de líquido entre las capas ayuda a reducir la fricción entre el corazón que late y los tejidos que lo rodean.
Algunas enfermedades pueden afectar el tejido cardíaco.""",
    'question': input("Pregunta: ")
})

# Imprimir solo la respuesta
print(result['answer'])

Pregunta: ¿Cuál es la función del miocardio?
es el saco que envuelve el corazón


In [ ]:

# Cargar el modelo entrenado y el tokenizador
model_name = './results'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Crear una muestra de entrada (una pregunta y un contexto)
inputs = tokenizer("¿Qué es la cardiopatía?", "La cardiopatía es un término general que describe una serie de trastornos del corazón.", return_tensors="pt")

# Convertir a ONNX
torch.onnx.export(
    model,  # El modelo entrenado
    (inputs["input_ids"], inputs["attention_mask"]),  # Entradas del modelo
    "bert_qa.onnx",  # Nombre del archivo de salida
    input_names=["input_ids", "attention_mask"],  # Nombres de las entradas
    output_names=["start_logits", "end_logits"],  # Nombres de las salidas
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_length"},
        "attention_mask": {0: "batch_size", 1: "seq_length"},
        "start_logits": {0: "batch_size", 1: "seq_length"},
        "end_logits": {0: "batch_size", 1: "seq_length"},
    },
    opset_version=14
)
